In [1]:
import pandas as pd
import json
import torch
from torch import nn

from settings.config import *

In [2]:
torch.cuda.is_available()

True

In [13]:

# for category in os.listdir(RAW_YUMMLY_IMAGES_PATH):
#     print(len(os.listdir(os.path.join(RAW_YUMMLY_IMAGES_PATH, category))))

path = YUMMLY_PATH
train_recipes_df = pd.DataFrame(json.load(open(os.path.join(path, 'train', METADATA_FILENAME))))
val_recipes_df = pd.DataFrame(json.load(open(os.path.join(path, 'val', METADATA_FILENAME))))
test_recipes_df = pd.DataFrame(json.load(open(os.path.join(path, 'test', METADATA_FILENAME))))

In [14]:
normalize = False

train_count = train_recipes_df['ingredients_ok'].explode().value_counts(normalize=normalize)
val_count = val_recipes_df['ingredients_ok'].explode().value_counts(normalize=normalize)
test_count = test_recipes_df['ingredients_ok'].explode().value_counts(normalize=normalize)

print(train_count, "\n\n")

ingredients_ok
salt              33985
oil               30007
pepper            26638
garlic            24494
onion             23404
                  ...  
cherry               79
tomate               77
chestnut             74
ancho chiles         61
serrano chiles       55
Name: count, Length: 182, dtype: int64 




#### 1: Use only the K most common ingredients (and remove recipes with less than 3 ingredients)


In [16]:
k = 100
common_ingredients = set(train_count.head(k).index)

In [17]:
train_recipes_k_common = train_recipes_df.copy()
train_recipes_k_common['ingredients_scraped'] = train_recipes_df['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
train_recipes_k_common



,course,cuisine,flavors,id,ingredients,ingredients_ok,name,old_id,image,ingredients_scraped
0,"[Soups, Appetizers, Lunch and Snacks]",French,"{'Bitter': 0.6667, 'Meaty': 0.6667, 'Piquant': 0.0, 'Salty': 0.8333, 'Sour': 0.1667, 'Sweet': 0.5}",0,"[6 cups chicken stock, 1/4 cup butter, 8 tablespoons all-purpose flour, 12 ounces Brie cheese, 3/8 cup white wine, 2 ounces julienned carrots, 1/4 cup chopped celery, 2 ounces fresh mushrooms, sliced, 1/4 cup heavy whipping cream, salt and pepper to taste, salt and pepper to taste]","[celery, cheese, cream, butter, chicken, carrot, salt, flour, wine, mushrooms]",Brie Soup,Brie-Soup-Allrecipes,000000____french.jpg,"[cream, cheese, chicken, wine, mushrooms, carrot, flour, celery, salt, butter]"
1,[Salads],Mexican,"{'Bitter': 0.3333, 'Meaty': 0.1667, 'Piquant': 0.5, 'Salty': 0.6667, 'Sour': 1.0, 'Sweet': 0.8333}",1,"[1 large jicama, peeled and cut into 1/4-inch-wide strips, 3 navel oranges, peeled and sectioned with pulp and membrane removed, 1 large cantaloupe or honeydew melon, peeled, seeded, and cut into bite-size chunks, 1/2 cup pomegranate seeds, 1 cup fresh lime juice, 2 sprigs cilantro, chopped, 1 teaspoon sea salt, 1/2 teaspoon chili powder (optional)]","[cheese, orange, salt, chili, juice]",Jícama-Melon Salad,JIcama_Melon-Salad-Epicurious,000001____mexican.jpg,"[juice, cheese, orange, chili, salt]"
2,"[Soups, Main Dishes]",Indian,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",2,"[ghee, sea bass fillets, lemon juice, cilantro leaves, black pepper, coarse sea salt, garam masala, sunflower oil, medium tomatoes, ground turmeric, tamarind pulp, unsalted butter, fresh curry leaves, onions, yellow mustard seeds, potatoes, garlic cloves, tomatoes, red chili peppers]","[tomato, butter, garam masala, pepper, cilantro leaves, curry, salt, ghee, garlic, potato, onion, oil, mustard, turmeric, lemon]",Sea Bass And Turmeric Potatoes In Rasam Broth,Sea-Bass-And-Turmeric-Potatoes-In-Rasam-Broth-1387398,000002____indian.jpg,"[turmeric, potato, onion, curry, lemon, pepper, tomato, garam masala, cilantro leaves, salt, butter, oil, mustard, garlic]"
3,[Main Dishes],Greek,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.16666666666666666, 'sour': 0.5, 'sweet': 0.16666666666666666}",3,"[olive oil, lemon juice, garlic, fresh oregano, chopped fresh thyme, chicken, red potato, onions, feta cheese crumbles, kalamata]","[cheese, oregano, garlic, chicken, potato, thyme, onion, oil, lemon]",Greek Roast Chicken,Greek-Roast-Chicken-1412352,000003____greek.jpg,"[potato, cheese, onion, lemon, thyme, chicken, oregano, oil, garlic]"
4,[Breakfast and Brunch],French,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",4,"[bananas, almond milk, cinnamon, nutmeg, vanilla extract, coconut oil, bread, sourdough loaf, pure maple syrup]","[banana, milk, vanilla, nutmeg, oil, syrup, cinnamon, bread]",Banana French Toast,Banana-French-Toast-897855,000004____french.jpg,"[cinnamon, milk, vanilla, nutmeg, syrup, bread, banana, oil]"
...,...,...,...,...,...,...,...,...,...,...
54719,[Soups],French,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.6666666666666666, 'sour': 0.16666666666666666, 'sweet': 0.16666666666666666}",54719,"[unsalted butter, olive oil, onions, beef broth, dry sherry, dried thyme, salt, pepper, french bread, provolone cheese, swiss cheese, grated parmesan cheese]","[cheese, butter, liquor, pepper, salt, beef, thyme, onion, oil, bread]",Rich and Simple French Onion Soup,Rich-and-Simple-French-Onion-Soup-1608385,054719____french.jpg,"[cheese, onion, thyme, pepper, liquor, bread, beef, salt, butter, oil]"
54720,[Main Dishes],Thai,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",54720,"[boneless skinless chicken breasts, mango, coconut milk, salt, red curry paste, curry powder, garlic powder, groun

In [18]:
# how many recipes would have < 3 ingredients left
train_recipes_k_common['tmp_len'] = train_recipes_k_common['ingredients_scraped'].apply(lambda x: len(x))
train_recipes_k_common = train_recipes_k_common[train_recipes_k_common['tmp_len'] >= 3].drop(columns=['tmp_len'])
print("Before: ", len(train_recipes_df), "After: ", len(train_recipes_k_common))

Before:  54724 After:  54267


In [19]:
val_recipes_k_common = val_recipes_df.copy()
val_recipes_k_common['ingredients_ok'] = val_recipes_df['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
val_recipes_k_common = val_recipes_k_common[val_recipes_k_common['ingredients_ok'].apply(lambda x: len(x) >= 3)]
print("Before: ", len(val_recipes_df), "After: ", len(val_recipes_k_common))

Before:  5210 After:  5150


In [20]:
test_recipes_k_common = test_recipes_df.copy()
test_recipes_k_common['ingredients_ok'] = test_recipes_df['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
test_recipes_k_common = test_recipes_k_common[test_recipes_k_common['ingredients_ok'].apply(lambda x: len(x) >= 3)]
print("Before: ", len(test_recipes_df), "After: ", len(test_recipes_k_common))

Before:  5212 After:  5167


In [21]:
#saving 
train_recipes_k_common = train_recipes_k_common.drop(columns=['ingredients_ok']).rename(columns={'ingredients_scraped': 'ingredients_ok'})


train_recipes_k_common.to_json(os.path.join(path, 'train', "topk_" + METADATA_FILENAME), orient='records', indent=4)
val_recipes_k_common.to_json(os.path.join(path, 'val', "topk_" + METADATA_FILENAME), orient='records', indent=4)
test_recipes_k_common.to_json(os.path.join(path, 'test', "topk_" + METADATA_FILENAME), orient='records', indent=4)

In [22]:
val_recipes_k_common

,course,cuisine,flavors,id,ingredients,ingredients_ok,name,old_id,image
1,[Main Dishes],Italian,"{'bitter': 0.3333333333333333, 'meaty': 0.16666666666666666, 'piquant': 0.16666666666666666, 'salty': 0.16666666666666666, 'sour': 0.5, 'sweet': 0.16666666666666666}",54725,"[olive oil, ground beef, sliced mushrooms, yellow onion, garlic, grated parmesan cheese, diced tomatoes, tomato sauce, tomato paste, italian seasoning, crushed red pepper flakes, bay leaves]","[italian seasoning, cheese, onion, tomato, pepper, bay leaves, mushrooms, beef, oil, garlic]",Slow Cooker Parmesan Meat Sauce,Slow-Cooker-Parmesan-Meat-Sauce-1587830,054725____italian.jpg
2,[Desserts],French,"{'bitter': 0.16666666666666666, 'meaty': 0.8333333333333334, 'piquant': 0.0, 'salty': 0.16666666666666666, 'sour': 0.16666666666666666, 'sweet': 0.5}",54726,"[heavy whipping cream, milk, vanilla beans, egg yolks, sugar, vanilla extract]","[sugar, cream, egg, milk, vanilla]",Vanilla Bean Creme Brulee,Vanilla-Bean-Creme-Brulee-514840,054726____french.jpg
3,[Salads],American,"{'Bitter': 0.1667, 'Meaty': 0.1667, 'Piquant': 0.0, 'Salty': 0.1667, 'Sour': 1.0, 'Sweet': 1.0}",54727,"[3/4 cup orange juice, 1/2 cup honey, 1 1/2 pints fresh strawberries, halved, 1 1/2 cups fresh raspberries, 1 1/2 cups fresh blueberries, 2 oranges, peeled and sectioned, 1 1/2 cups honeydew melon balls, 1/3 cup mint leaves, chopped]","[mint, orange, honey]",Favorite Fruit Salad,Favorite-Fruit-Salad-My-Recipes,054727____american.jpg
4,[Appetizers],Mexican,"{'bitter': 0.3333333333333333, 'meaty': 0.16666666666666666, 'piquant': 0.8333333333333334, 'salty': 0.16666666666666666, 'sour': 0.6666666666666666, 'sweet': 0.16666666666666666}",54728,"[halibut fillets, fresh lime juice, white onion, roma tomatoes, jalapeno chilies, green olives, chopped cilantro, extra-virgin olive oil, salt, sugar, avocado, tortilla chips]","[sugar, juice, onion, tomato, avocado, chili, salt, oil, cilantro]",Ceviche,Ceviche-1171665,054728____mexican.jpg
5,[Salads],American,"{'Bitter': 0.3333, 'Meaty': 0.1667, 'Piquant': 0.0, 'Salty': 0.3333, 'Sour': 0.8333, 'Sweet': 0.3333}",54729,"[4 tablespoons olive oil, divided, 2 tablespoons red wine vinegar, 1 small shallot, finely chopped, 1 tablespoon chopped fresh parsley, 1 teaspoon Dijon mustard, 1 teaspoon chopped fresh thyme, 6 ounces thinly sliced prosciutto, cut crosswise into 1/3-inch-wide strips (about 1 1/2 cups), 1 lemon, halved, plus 2 tablespoons fresh lemon juice, 4 large artichokes, 1 large fennel bulb with fronds, bulb halved lengthwise and very thinly sliced crosswise, fronds chopped, 1 large fennel bulb with fronds, bulb halved lengthwise and very thinly sliced crosswise, fronds chopped, 2 medium heads of frisée, torn into bite-size pieces]","[shallots, lemon, thyme, wine, parsley, oil, mustard]","Artichoke, Fennel, and Crispy Prosciutto Salad",Artichoke_-Fennel_-And-Crispy-Prosciutto-Salad-Epicurious,054729____american.jpg
...,...,...,...,...,...,...,...,...,...
5205,[Beverages],Greek,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",59929,"[bananas, baby spinach, peanut butter, unsweetened vanilla almond milk, fat free greek yogurt]","[milk, spinach, butter, banana, yogurt]",Skinny Green Monster Smoothie,Skinny-green-monster-smoothie-351929,059929____greek.jpg
5206,[Desserts],American,"{'Bitter': 0.1667, 'Meaty': 0.1667, 'Piquant': 0.0, 'Salty': 0.1667, 'Sour': 0.1667, 'Sweet': 0.6667}",59930,"[Bread Pudding, 1 Granny Smith apple, peeled and chopped, 1/2 tsp ground cinnamon, divided, 1/2 Italian bread loaf, cut into bite-size pieces, Vegetable cooking spray, 3 large eggs, 1 1/2 cups reduced-fat milk, 1 cup apple cider, 1/4 cup firmly packed brown sugar, 1 teaspoon vanilla extract, 1/4 teaspoon ground nutmeg, Toasted Pecan Caramel Sauce, 1/4 cup chopped pecans, 3/4 cup sugar, 1 teaspoon light corn syrup, 1/2 cup evaporated milk, 1 1/2 teaspoon butter]","[sugar, egg, cinnamon, corn, milk, pecan, vanilla, nutmeg, bread, 

# TMP


In [19]:
train_recipes_df_tmp = train_recipes_df.sample(1000)[['ingredients', 'ingredients_ok']]
train_recipes_df_tmp.to_csv(os.path.join(path, 'train', "tmp.csv"))

In [57]:
from prev_attempts.attempt2.utils import preprocess_ingredient

train_recipes_tmp = train_recipes_df.copy()
train_recipes_tmp['ingredients_ok'] = train_recipes_df['ingredients'].apply(lambda x: [preprocess_ingredient(ing) for ing in x])
train_recipes_tmp

,course,cuisine,flavors,id,ingredients,ingredients_ok,name,old_id,image
0,"[Soups, Appetizers, Lunch and Snacks]",French,"{'Bitter': 0.6667, 'Meaty': 0.6667, 'Piquant': 0.0, 'Salty': 0.8333, 'Sour': 0.1667, 'Sweet': 0.5}",0,"[6 cups chicken stock, 1/4 cup butter, 8 tablespoons all-purpose flour, 12 ounces Brie cheese, 3/8 cup white wine, 2 ounces julienned carrots, 1/4 cup chopped celery, 2 ounces fresh mushrooms, sliced, 1/4 cup heavy whipping cream, salt and pepper to taste, salt and pepper to taste]","[chicken stock, butter, all-purpose flour, brie cheese, white wine, julienned carrots, celery, mushrooms, whipping cream, salt and pepper to taste, salt and pepper to taste]",Brie Soup,Brie-Soup-Allrecipes,000000____french.jpg
1,[Salads],Mexican,"{'Bitter': 0.3333, 'Meaty': 0.1667, 'Piquant': 0.5, 'Salty': 0.6667, 'Sour': 1.0, 'Sweet': 0.8333}",1,"[1 large jicama, peeled and cut into 1/4-inch-wide strips, 3 navel oranges, peeled and sectioned with pulp and membrane removed, 1 large cantaloupe or honeydew melon, peeled, seeded, and cut into bite-size chunks, 1/2 cup pomegranate seeds, 1 cup fresh lime juice, 2 sprigs cilantro, chopped, 1 teaspoon sea salt, 1/2 teaspoon chili powder (optional)]","[jicama, and cut into -inch-wide strips, navel oranges, and sectioned with pulp and membrane removed, cantaloupe or honeydew melon, , seeded, and cut into bite-size chunks, pomegranate seeds, lime juice, sprigs cilantro, sea salt, chili powder]",Jícama-Melon Salad,JIcama_Melon-Salad-Epicurious,000001____mexican.jpg
2,"[Soups, Main Dishes]",Indian,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",2,"[ghee, sea bass fillets, lemon juice, cilantro leaves, black pepper, coarse sea salt, garam masala, sunflower oil, medium tomatoes, ground turmeric, tamarind pulp, unsalted butter, fresh curry leaves, onions, yellow mustard seeds, potatoes, garlic cloves, tomatoes, red chili peppers]","[ghee, sea bass fillets, lemon juice, cilantro leaves, black pepper, sea salt, garam masala, sunflower oil, tomatoes, turmeric, tamarind pulp, butter, curry leaves, onions, yellow mustard seeds, potatoes, garlic cloves, tomatoes, red chili peppers]",Sea Bass And Turmeric Potatoes In Rasam Broth,Sea-Bass-And-Turmeric-Potatoes-In-Rasam-Broth-1387398,000002____indian.jpg
3,[Main Dishes],Greek,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.16666666666666666, 'sour': 0.5, 'sweet': 0.16666666666666666}",3,"[olive oil, lemon juice, garlic, fresh oregano, chopped fresh thyme, chicken, red potato, onions, feta cheese crumbles, kalamata]","[olive oil, lemon juice, garlic, oregano, thyme, chicken, red potato, onions, feta cheese crumbles, kalamata]",Greek Roast Chicken,Greek-Roast-Chicken-1412352,000003____greek.jpg
4,[Breakfast and Brunch],French,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",4,"[bananas, almond milk, cinnamon, nutmeg, vanilla extract, coconut oil, bread, sourdough loaf, pure maple syrup]","[bananas, almond milk, cinnamon, nutmeg, vanilla extract, coconut oil, bread, sourdough loaf, pure maple syrup]",Banana French Toast,Banana-French-Toast-897855,000004____french.jpg
...,...,...,...,...,...,...,...,...,...
54719,[Soups],French,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.6666666666666666, 'sour': 0.16666666666666666, 'sweet': 0.16666666666666666}",54719,"[unsalted butter, olive oil, onions, beef broth, dry sherry, dried thyme, salt, pepper, french bread, provolone cheese, swiss cheese, grated parmesan cheese]","[butter, olive oil, onions, beef broth, sherry, thyme, salt, pepper, french bread, provolone cheese, swiss cheese, parmesan cheese]",Rich and Simple French Onion Soup,Rich-and-Simple-French-Onion-Soup-1608385,054719____french.jpg
54720,[Main Dishes],Thai,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",54720,"[boneless skin

In [58]:

train_count = train_recipes_tmp['ingredients_ok'].explode().value_counts(normalize=normalize)
train_count

ingredients_ok
salt                                                     21774
garlic                                                   10822
olive oil                                                10561
water                                                     9209
sugar                                                     9151
                                                         ...  
baby artichokes or  artichoke hearts                         1
bite-size pieces  and ed frisee or salad mix,  and ed        1
bunches green onion                                          1
salt -oon                                                    1
of whole milk ricotta cheese                                 1
Name: count, Length: 54329, dtype: int64

In [59]:
k = 10
common_ingredients = set(train_count.head(k).index)
common_ingredients

{'black pepper',
 'butter',
 'eggs',
 'garlic',
 'ginger',
 'olive oil',
 'pepper',
 'salt',
 'sugar',
 'water'}

In [60]:
train_recipes_tmp['ingredients_scraped'] = train_recipes_tmp['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
train_recipes_tmp['ingredients_scraped'] = train_recipes_tmp['ingredients_scraped'].apply(lambda x: [ing.replace(" ", "_") for ing in x if ing != ''])

train_recipes_tmp



,course,cuisine,flavors,id,ingredients,ingredients_ok,name,old_id,image,ingredients_scraped
0,"[Soups, Appetizers, Lunch and Snacks]",French,"{'Bitter': 0.6667, 'Meaty': 0.6667, 'Piquant': 0.0, 'Salty': 0.8333, 'Sour': 0.1667, 'Sweet': 0.5}",0,"[6 cups chicken stock, 1/4 cup butter, 8 tablespoons all-purpose flour, 12 ounces Brie cheese, 3/8 cup white wine, 2 ounces julienned carrots, 1/4 cup chopped celery, 2 ounces fresh mushrooms, sliced, 1/4 cup heavy whipping cream, salt and pepper to taste, salt and pepper to taste]","[chicken stock, butter, all-purpose flour, brie cheese, white wine, julienned carrots, celery, mushrooms, whipping cream, salt and pepper to taste, salt and pepper to taste]",Brie Soup,Brie-Soup-Allrecipes,000000____french.jpg,[butter]
1,[Salads],Mexican,"{'Bitter': 0.3333, 'Meaty': 0.1667, 'Piquant': 0.5, 'Salty': 0.6667, 'Sour': 1.0, 'Sweet': 0.8333}",1,"[1 large jicama, peeled and cut into 1/4-inch-wide strips, 3 navel oranges, peeled and sectioned with pulp and membrane removed, 1 large cantaloupe or honeydew melon, peeled, seeded, and cut into bite-size chunks, 1/2 cup pomegranate seeds, 1 cup fresh lime juice, 2 sprigs cilantro, chopped, 1 teaspoon sea salt, 1/2 teaspoon chili powder (optional)]","[jicama, and cut into -inch-wide strips, navel oranges, and sectioned with pulp and membrane removed, cantaloupe or honeydew melon, , seeded, and cut into bite-size chunks, pomegranate seeds, lime juice, sprigs cilantro, sea salt, chili powder]",Jícama-Melon Salad,JIcama_Melon-Salad-Epicurious,000001____mexican.jpg,[]
2,"[Soups, Main Dishes]",Indian,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",2,"[ghee, sea bass fillets, lemon juice, cilantro leaves, black pepper, coarse sea salt, garam masala, sunflower oil, medium tomatoes, ground turmeric, tamarind pulp, unsalted butter, fresh curry leaves, onions, yellow mustard seeds, potatoes, garlic cloves, tomatoes, red chili peppers]","[ghee, sea bass fillets, lemon juice, cilantro leaves, black pepper, sea salt, garam masala, sunflower oil, tomatoes, turmeric, tamarind pulp, butter, curry leaves, onions, yellow mustard seeds, potatoes, garlic cloves, tomatoes, red chili peppers]",Sea Bass And Turmeric Potatoes In Rasam Broth,Sea-Bass-And-Turmeric-Potatoes-In-Rasam-Broth-1387398,000002____indian.jpg,"[butter, black_pepper]"
3,[Main Dishes],Greek,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.16666666666666666, 'sour': 0.5, 'sweet': 0.16666666666666666}",3,"[olive oil, lemon juice, garlic, fresh oregano, chopped fresh thyme, chicken, red potato, onions, feta cheese crumbles, kalamata]","[olive oil, lemon juice, garlic, oregano, thyme, chicken, red potato, onions, feta cheese crumbles, kalamata]",Greek Roast Chicken,Greek-Roast-Chicken-1412352,000003____greek.jpg,"[olive_oil, garlic]"
4,[Breakfast and Brunch],French,"{'bitter': None, 'meaty': None, 'piquant': None, 'salty': None, 'sour': None, 'sweet': None}",4,"[bananas, almond milk, cinnamon, nutmeg, vanilla extract, coconut oil, bread, sourdough loaf, pure maple syrup]","[bananas, almond milk, cinnamon, nutmeg, vanilla extract, coconut oil, bread, sourdough loaf, pure maple syrup]",Banana French Toast,Banana-French-Toast-897855,000004____french.jpg,[]
...,...,...,...,...,...,...,...,...,...,...
54719,[Soups],French,"{'bitter': 0.16666666666666666, 'meaty': 0.16666666666666666, 'piquant': 0.0, 'salty': 0.6666666666666666, 'sour': 0.16666666666666666, 'sweet': 0.16666666666666666}",54719,"[unsalted butter, olive oil, onions, beef broth, dry sherry, dried thyme, salt, pepper, french bread, provolone cheese, swiss cheese, grated parmesan cheese]","[butter, olive oil, onions, beef broth, sherry, thyme, salt, pepper, french bread, provolone cheese, swiss cheese, parmesan cheese]",Rich and Simple French Onion Soup,Rich-and-Simple-French-Onion-Soup-1608385,054719____french.jpg,"[olive_oil, salt, butter, pepper]"
54720,[Main Dishes]

In [61]:
# how many recipes would have < 3 ingredients left
train_recipes_tmp['tmp_len'] = train_recipes_tmp['ingredients_scraped'].apply(lambda x: len(x))
train_recipes_tmp = train_recipes_tmp[train_recipes_tmp['tmp_len'] >= 3].drop(columns=['tmp_len'])
print("Before: ", len(train_recipes_df), "After: ", len(train_recipes_tmp))
train_recipes_tmp = train_recipes_tmp.drop(columns=['ingredients_ok']).rename(columns={'ingredients_scraped': 'ingredients_ok'})
train_recipes_tmp.to_json(os.path.join(path, 'train', "tmp_topk_" + METADATA_FILENAME), orient='records', indent=4)


Before:  54724 After:  15907


In [62]:
val_recipes_tmp = val_recipes_df.copy()
val_recipes_tmp['ingredients_ok'] = val_recipes_df['ingredients'].apply(lambda x: [preprocess_ingredient(ing) for ing in x])
val_recipes_tmp['ingredients_ok'] = val_recipes_df['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
val_recipes_tmp = val_recipes_tmp[val_recipes_tmp['ingredients_ok'].apply(lambda x: len(x) >= 3)]

val_recipes_tmp['ingredients_ok'] = val_recipes_tmp['ingredients_ok'].apply(lambda x: [ing.replace(" ", "_") for ing in x if ing != ''])
train_recipes_tmp.to_json(os.path.join(path, 'val', "tmp_topk_" + METADATA_FILENAME), orient='records', indent=4)
print("Before: ", len(val_recipes_df), "After: ", len(val_recipes_tmp))


Before:  5210 After:  2218


In [63]:
test_recipes_tmp = test_recipes_df.copy()
test_recipes_tmp['ingredients_ok'] = test_recipes_df['ingredients'].apply(lambda x: [preprocess_ingredient(ing) for ing in x])
test_recipes_tmp['ingredients_ok'] = test_recipes_df['ingredients_ok'].apply(lambda x: list(set(x).intersection(common_ingredients)))
test_recipes_tmp = test_recipes_tmp[test_recipes_tmp['ingredients_ok'].apply(lambda x: len(x) >= 3)]

test_recipes_tmp['ingredients_ok'] = test_recipes_tmp['ingredients_ok'].apply(lambda x: [ing.replace(" ", "_") for ing in x if ing != ''])
test_recipes_tmp.to_json(os.path.join(path, 'test', "tmp_topk_" + METADATA_FILENAME), orient='records', indent=4)
print("Before: ", len(test_recipes_df), "After: ", len(test_recipes_tmp))

Before:  5212 After:  2215


#### 2: Ingredients embeddings 

